## Q1.

Add methods `__iter__` to your project Time Series class to iterate over values, a method `itertimes` to iterate over times, a method `itervalues` to iterate over values, and a method `iteritems` to iterate over time-value pairs. (This is a similar interface to python dictionaries). To test these, check both the types of the results and the answers you expect.

In [1]:
#code in the project repo

## Q2.

An online mean and standard deviation algorithm.

Below is a function to generate a potentially infinite stream of 1-D data.

In [2]:
from random import normalvariate, random
from itertools import count
def make_data(m, stop=None):
    for _ in count():
        if stop and _ > stop:
            break
        yield 1.0e09 + normalvariate(0, m*random() )
        

Here is an implementation of an online mean algorithm..see http://www.johndcook.com/blog/standard_deviation/ and the link to http://www.johndcook.com/blog/2008/09/26/comparing-three-methods-of-computing-standard-deviation/ in-between. (Convince yourselves of the formulas...)

In [3]:
def online_mean(iterator):
    n = 0
    mu = 0
    for value in iterator:
        n += 1
        delta = value - mu
        mu = mu + delta/n
        yield mu

We use out generator functions to implement iterators:

In [4]:
g = make_data(5, 10)
list(g)

[1000000003.5790623,
 1000000000.6541448,
 1000000005.9628466,
 999999999.5840406,
 1000000001.9734354,
 1000000001.4752544,
 1000000000.7058123,
 999999999.9354751,
 1000000003.5085111,
 1000000000.4493423,
 999999994.7385846]

In [5]:
g = online_mean(make_data(5, 100))
print(type(g))
list(g)

<class 'generator'>


[999999998.5913218,
 1000000000.230273,
 999999999.8482201,
 1000000000.3377349,
 1000000000.1823242,
 1000000000.2242544,
 1000000000.1807829,
 999999999.8194172,
 999999999.8534858,
 999999999.8527277,
 999999999.6223774,
 999999999.6321985,
 999999999.8797985,
 999999999.894194,
 999999999.8245207,
 999999999.7832695,
 999999999.7832966,
 999999999.755147,
 999999999.767698,
 999999999.8211688,
 999999999.8582386,
 999999999.8709852,
 999999999.8583456,
 999999999.8048066,
 999999999.746259,
 999999999.8718702,
 999999999.89138,
 999999999.9800614,
 999999999.968699,
 1000000000.1173884,
 1000000000.1448383,
 1000000000.101137,
 1000000000.0005,
 1000000000.0075909,
 1000000000.0342863,
 999999999.9990256,
 999999999.9863507,
 999999999.9468405,
 999999999.9712906,
 1000000000.1306854,
 1000000000.1747465,
 1000000000.1685058,
 1000000000.1647862,
 1000000000.1191261,
 1000000000.1547035,
 1000000000.1408533,
 1000000000.1156892,
 1000000000.0874469,
 1000000000.0846573,
 1000000000

### 2.1

Implement the standard deviation algorithm as a generator function as

```python
def online_mean_dev(iterator):
    BLA BLA
    if n > 1:
        stddev = math.sqrt(dev_accum/(n-1))
        yield (n, value, mu, stddev)
```

In [6]:
# your code here
import math
def online_mean_dev(iterator):
    n = 0
    mu = 0
    mu_1 = 0
    dev_accum = 0
    for value in iterator:
        n += 1
        mu_1 = mu
        mu += (value - mu)/n
        dev_accum += (value - mu_1)*(value - mu)
        if n > 1:
            stddev = math.sqrt(dev_accum/(n-1))
            yield (n, value, mu, stddev)

Here we make 100000 element data, and run this iterator on it (imagine running this on a time-series being slowly read from disk

In [7]:
data_with_stats = online_mean_dev(make_data(5, 100000))

## Q3.

Let's do Anomaly detection. Write a routine `is_ok`:

```python
def is_ok(level, t)
```

which takes a tuple like the one yielded by your code above and returns True if the value is inbetween `level`-$\sigma$ of the mean.

In [8]:
#your code here
#level - sigma or abs(level-sigma)?
def is_ok(level,t):
    (_, val, mu, stddev) = t
    return level - stddev >= abs(mu-val)

We use this function to create a predicate passed through to `itertools.filterfalse` which is then used to obtain an iterator on the anomalies.

In [9]:
from itertools import filterfalse
pred = lambda t: is_ok(5, t)
anomalies = filterfalse(pred, data_with_stats)

We materialize the anomalies...

In [10]:
list(anomalies)#materialize

[(4, 999999995.0468718, 999999998.6314737, 2.422199524800372),
 (6, 999999989.6497024, 999999997.530956, 4.396921201482492),
 (7, 1000000000.2008059, 999999997.9123632, 4.138727235792861),
 (8, 1000000000.9275091, 999999998.2892565, 3.9772405248257123),
 (15, 1000000002.8425807, 999999998.863884, 3.1412133341991137),
 (17, 1000000004.6202906, 999999999.1811415, 3.2567047751306784),
 (24, 1000000003.1391132, 999999999.4715954, 2.866382612299841),
 (27, 1000000002.9965852, 999999999.6449134, 2.7821754230415734),
 (33, 1000000002.3009864, 999999999.5658511, 2.598933427940671),
 (34, 999999995.8363676, 999999999.4561604, 2.637965908010402),
 (41, 1000000005.2237408, 999999999.6666762, 2.569344897278922),
 (47, 1000000003.3874924, 999999999.7877169, 2.468930369298502),
 (49, 1000000002.546547, 999999999.8178873, 2.4563037618168364),
 (58, 999999995.4919074, 999999999.7456892, 2.3819676831879213),
 (61, 999999990.8553153, 999999999.5977912, 2.587294835431414),
 (66, 1000000002.9406499, 99999

## To think of, but not hand in

What kinds of anomalies will this algorithm pick up? What kinds would a shorter "window" of anomaly detection, like 100 points around the time in question pick? How might you create an algorithm which does window based averaging? (hint: the window size is small compared to the time series size). 

Finally think a bit of how you might implement all of this in a production environment..remember that data streaming in might get backed up when you handle an anomaly.

(Some inspiration might accrue if you look at the docs for `collections.deque`).

If there's no semblance of std dev in the anomaly detection, then you're finding values outside of your level parameter. Incorporating the std dev adds a penalty for the noise of the data, for example if std dev exceeds the level then the level-stddev value goes negative and everything's considered an anomaly. On the flip side if the variance of the data is very small then no values will exceed the level parameter. For window averaging just create n simultaneous averaging variables for an n-window period, once one reaches n it resets. Not sure what issue you're getting at for production environment - if data stream backed up your algorithm just halts waiting for the data